<img src="Imagenes/logo.png" alt="IPython Notebook" style="width: 200px;"/>

***

<p style= "text-align:center; font-family: Calibri; font-size: x-large; color: black; font-weight:bold;"> DETECCIÓN DE TEXTO EN ARCHIVO, PARA LA VERIFICACIÓN DE VIDA LABORAL Y ANTECEDENTES DE CUIDADORES DE IBERASIS </p>

<p style= "text-align:center; font-family: Calibri; font-size: medium; color: black; font-weight:bold;"> Julieta Benítez │ PROYECTO FINAL A4 | PBD1 </p>

En este Notebook se realizará la extracción de datos de un documento correspondiente con la vida laboral de una persona, código que se utilizará de forma automática en la aplicaciónde IberAsis, el cual tiene como objetivo la selección de perfiles para el sector geriátrico, dependencia y niños en tiempo real, mediante la verificación de antiguos empleadores y la comprobación de vida laboral y antecedentes.

IberAsis comprobará la seguridad de los posibles cuidadores, verificando el **informe de vida laboral** y el **certificado de antecedentes penales**, mediante el código CEA en el primer caso, y el código seguro de verificación en el último caso. Para ambos sucesos se deberá extraer primeramente los datos del archivo pdf en cuestión, que es lo que se ha realizado a continuación, utilizando OCR de Google Cloud Vision:

In [1]:
#Importamos librerías necesarias
import os, io
import re
from google.cloud import vision
from google.cloud import storage #Conecta con el storage de nuestro GCP
from google.protobuf import json_format #Módulo de formato JSON

In [2]:
#Token JSON File, que corresponde a la Key creada en las credenciales 
#relacionadas con la API de Cloud Vision
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'ServiceAccountToken.json'

<img src="Imagenes/Key_API_Vision.png" alt="IPython Notebook"/>

In [3]:
#ImageAnnotatorCliente es el servicio que realiza tareas de detección de la 
#API de Google Cloud Vision. En este caso queremos detectar texto de un 
#archivo pdf
client = vision.ImageAnnotatorClient()

In [4]:
#Creamos un tamaño de lote = 2 para nuestros resultados
batch_size = 2
mime_type = 'application/pdf' #Naturaleza y formato del documento

#A continuación construimos una instancia tipo Feature de cloud Vision,
#concretamente la detección de texto en documentos
feature = vision.types.Feature( 
    type=vision.enums.Feature.Type.DOCUMENT_TEXT_DETECTION)

In [5]:
#Indicamos dónde se encuentra el archivo que deseamos analizar,
#el cual se encuentra en el storage de GCP, concretamente en el 
#bucket vidaslaborales_iberasis con nombre vidalaboral.pdf

#La url del archivo lo encontramos en los detalles del mismo en GCS
gcs_source_uri = 'gs://vidaslaborales_iberasis/vidalaboral.pdf'
gcs_source = vision.types.GcsSource(uri=gcs_source_uri)
input_config = vision.types.InputConfig(gcs_source=gcs_source, mime_type=mime_type)

<img src="Imagenes/GCS.png" alt="IPython Notebook" />

In [6]:
#El resultado de la detección de texto del PDF relacionado con la vida laboral
#se guardará en el mismo bucket y con formato JSON
gcs_destination_uri = 'gs://vidaslaborales_iberasis/resultados '
gcs_destination = vision.types.GcsDestination(uri=gcs_destination_uri)
output_config = vision.types.OutputConfig(gcs_destination=gcs_destination, batch_size=batch_size)

In [7]:
#Creamos un request con un objeto tipo input config
async_request = vision.types.AsyncAnnotateFileRequest(
    features=[feature], input_config=input_config, output_config=output_config)

In [8]:
#Creamos nuestro código de operación con un tiempo de 3 minutos
operation = client.async_batch_annotate_files(requests=[async_request])
operation.result(timeout=180)

responses {
  output_config {
    gcs_destination {
      uri: "gs://vidaslaborales_iberasis/resultados "
    }
    batch_size: 2
  }
}

In [9]:
#API Google Cloud Storage
storage_client = storage.Client()
#Seperación de elementos en diferentes grupos
match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)
bucket_name = match.group(1) #Grupo 1
prefix = match.group(2) #Grupo 2
bucket = storage_client.get_bucket(bucket_name) #Buscamos el archivo que hemos
                                                #usado para realizar la 
                                                #operación

In [10]:
#En GCS se utiliza el término blob para identificar el output que se ha
#generado utilizando la API, el cual corresponde a un objeto de lsita con el
#prefijo dado
blob_list = list(bucket.list_blobs(prefix=prefix))
print('Output files:')
for blob in blob_list:
    print(blob.name)

Output files:
resultados output-1-to-2.json
resultados output-3-to-4.json


Observamos que nuestro programa nos devuelve 2 archivos json con la información extraída del pdf relacionado con la vida laboral:

<img src="Imagenes/Output.png" alt="IPython Notebook"/>

In [12]:
#Creamos un objeto de respuesta que utiliza el método 
#AnnotateFileResponse para generar una salida del objeto 
#en formato json
output = blob_list[0]
json_string = output.download_as_string()
response = json_format.Parse(
            json_string, vision.types.AnnotateFileResponse())

In [13]:
#Del objeto response, declaramos la posición 0 del mismo como 
#first_page_response ya que corresponde con la primera página del pdf
first_page_response = response.responses[0]
#full_text_annotation nos devuelve las entidades detectadas del documento
annotation = first_page_response.full_text_annotation

In [14]:
#Finalmente, obserevamos el texto detectado resultante de la primera página
print(annotation.text)

GOBIERNO
DE ESPAÑA
MINISTERIO
DE EMPLEO
Y SEGURIDAD SOCIAL
E
TESORERÍA GENERAL
DE LA SEGURIDAD SOCIAL
INFORME DE
VIDA LABORAL
De los antecedentes obrantes en la Tesorería General de la Seguridad Social al día 30 de Abril de 2015, resulta que D/Da JOSE JAVIER ABELLAN
MONTES, nacido/a el 29 de Enero de 1987, con Número de la Seguridad Social 081136091146, D.N.I. 038853816P, domicilio en CALLE OLIVAR N° 51
PISO 5 PTA. 5, 08350 ARENYS DE MAR BARCELONA ha figurado en situación de alta en el Sistema de la Seguridad Social durante un total de
2
Años
766
días
1
Meses
5
Días
Presenta las situaciones que se relacionan en las sucesivas hojas del presente informe.
Cualquier duda o aclaración sobre este informe le será atendida en el teléfono 901 50 20 50, en la web www.seg-social.es o cualquier Administración de la Seguridad Social.
La información sobre las situaciones indicadas no comprende ni los datos relativos a los Regímenes Especiales de los Funcionarios Civiles del Estado, de las
Fuerzas Ar

Observamos que nuestro programa ha detectado correctamente la información del documento relacionado con la vida laborable de una persona. El dato que nos interesará a la hora de verificar el documento será el código CEA, el cual recordemos que será comprobado mediante el servicio de verificación de integridad documental de la Seguridad Social utilizando Web Scraping con Selenium, una herramienta que permite utilizar un navegador web como si fuese un humano. En el caso del certificado de antecedentes penales, se utilizará también el web scraping con Selenium, esta vez con el servicio de verificación de certificados de Antecedentes Penales y Delitos de Naturaleza Sexual del Ministerio de Justicia, mediante el código seguro de verificación antes mencionado.

El código CEA identificado mediante google cloud vision ha resultado en: GMPT4-NR3KB-G5Z6M-K3ALR-ALZB2-4ZTAP. A continuación, observamos una imagen del documento que hemos examinado, para comprobar que efectivamente, se ha extraído el código CEA correcto:

<img src="Imagenes/CodigoCEA.png" alt="IPython Notebook" style="width: 500px;"/>

***

<img src="Imagenes/ue.png" alt="IPython Notebook" style="width: 300px;"/>